# Sweeps - Capacitance matrix

### Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

In [ ]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs.

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '5.7 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2 Metal passes information to the simulator "q3d" to extract the capacitance matrix.


In [ ]:
c1 = LOManalysis(design, "q3d")

Prepare data to pass as arguments for method run_sweep().

In [ ]:
render_design_argument_qcomps = ['Q1']

render_design_argument_endcaps = [('Q1', 'readout'), ('Q1', 'bus1'),('Q1', 'bus2')]


In [ ]:
# To identify the agruments that you can change.
# They will change based on the simulation software used.
c1.sim.setup

In [ ]:
# For the simulation software, if you don't want to use the default values, 
# you can update them as seen below.

# If a setup named "sweeper_q3d_setup" exists in the project, it will be deleted, 
# and a new setup will be added.
c1.sim.setup.name = "sweeper_q3d_setup"

c1.sim.setup.freq_ghz = 5.6
c1.sim.setup.max_passes = 9
c1.sim.setup.min_passes = 2
c1.sim.setup.percent_error = 0.45



We will look at modifying the pad_gap of qubit 1, to see how it impacts the anharmonicity of the qubit.

The steps will be;
- Connect to Ansys Q3D.
- Rebuild QComponents in Metal.
- Render QComponents within Q3D and setup the simulation.
- Delete/Clear the Q3D between each simulation.
- Using the capacitance matrices, LOM for each value in option_sweep is found.

#### Returns a dict and return code.  If the return code is zero, there were no errors detected.  
#### The dict has:  key = each value used to sweep, value = capacitance matrix

#### This could take minutes or hours based on the complexity of the design.



In [ ]:
sweep_data, return_code = c1.run_sweep(q1.name,
                                       'pad_gap',
                                       ['20um', '30um', '40um'],
                                       render_design_argument_qcomps,
                                       render_design_argument_endcaps,
                                       design_name="GetCapacitance",
                                       box_plus_buffer=True)

In [ ]:
from pandas import DataFrame

ec_val = []
for opt_val in sweep_data.keys():
    ec_val.append([opt_val,sweep_data[opt_val]['variables']['lumped_oscillator']['EC']])

df=DataFrame(ec_val,columns = ['Sweep Value', 'Ec'])
df

We can grab specific values from the results as seen below;

In [ ]:
sweep_data.keys()

In [ ]:
# For each value of option, there is a set of data.
sweep_data['20um'].keys()

In [ ]:
sweep_data['20um']['variables'].keys()

In [ ]:
sweep_data['20um']['sim_variables'].keys()

A boolean flag indicating whether a sweep has converged can be obtained from the `is_converged` key within the `sim_variables`. For instance, the following code retrieves the convergence result for the sweep parameter, 20um. The convergence is extracted from the Ansys Q3D renderer.

In [ ]:
sweep_data['20um']['sim_variables']['is_converged']

In [ ]:
if return_code ==0:
    print(f'{sweep_data.keys()} \n')
    for key in sweep_data.keys():
        print(f'\nkey={key}')
        
        option_name = sweep_data[key]['option_name']
        print(f'option_name[\'{key}\'][\'option_name\']={option_name}')
        
    
        variables = sweep_data[key]['variables']
        sim_variables = sweep_data[key]['sim_variables']
        
        print(f'variables={variables}')
        print(f'sim_variables={sim_variables}')
        

        

# READ THIS BEFORE running the cell.
This cell is to demonstrate that if you have already executed c1.sim.run(), you don't have to set up 
the environment again for c1.run_sweep().  In another words, if you don't pass updated arguments to
c1.run_sweep(), then c1.run_sweep() looks for the previous desgin arguments. 

If you pass anything more than these three arguments: qcomp_name, option_name, option_sweep ..... 
Then NOTHING will be used from previous run.  
```
c1.sim.solution_order = 'Medium'
c1.sim.auto_increase_solution_order = 'False'


c1.sim.run(components=render_design_argument_qcomps,
           open_terminations=render_design_argument_endcaps)
```

Because c1.sim.setup.run has the information from last run, this is OK.

```
sweep_data, return_code = c1.run_sweep(q1.name, 
                                        'pad_gap', 
                                        ['20um', '30um', '40um'])
```

In [ ]:
c1.sim.close()

In [ ]:
# Uncomment next line if you would like to close the gui
gui.main_window.close()